In [ ]:
using Documenter;
using Crayons;
using Logging;
using JuMP, MathOptInterface;
using Gurobi;
const MOI = MathOptInterface;
const MOIU = MOI.Utilities;

# LP Knapsack Problem

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} \le C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [ ]:
# Model definition
# References:
# [1] https://jump.dev/MathOptInterface.jl/stable/tutorials/example/

model = MOIU.Model{Float64}()

n = 3;
c = [1.0, 2.0, 3.0];  #  6.0
w = [3.0, 5.0, 10.0]; # 18.0 (32.0 - 18.0 = 14.0) (expansion = 2.0 + 4.0 + 8.0 = 14.0)
C = 32.0;

x = MOI.add_variables(model, n);

# ---------
# Objective
# ---------
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)

MOI.set(
   model,
   MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0),
);

# -----------
# Constraints
# -----------
MOI.add_constraint(
   model,
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0),
   MOI.LessThan(C),
);

for xᵢ in x
   MOI.add_constraint(model, xᵢ, MOI.ZeroOne())
end

In [ ]:
success() = println(crayon"#66ff66", "QUBO Success", crayon"#ffffff");
failure() = println(crayon"#ff6666", "QUBO Failure", crayon"#ffffff");

In [ ]:
include("../src/ToQUBO.jl")

qubo = ToQUBO.toqubo(model)

ToQUBO.isqubo(qubo) ? success() : failure();

print(qubo.model);

gurobi = Gurobi.Optimizer();
varmap = MOI.copy_to(gurobi, qubo.model);
MOI.optimize!(gurobi);

for xᵢ in MOI.get(gurobi, MOI.ListOfVariableIndices())
    vᵢ = varmap[xᵢ]
    if ToQUBO.isslack(qubo.target[vᵢ])
        var = "s"
    else
        var = "x"
    end
    println(ToQUBO.subscript(vᵢ, var=var), " = ", MOI.get(gurobi, MOI.VariablePrimal(), xᵢ));
end